In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...
Optimize a model with 314 rows, 314 columns and 314 nonzeros
Model has 49455 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e-03, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 314 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 314 columns, 0 nonzeros
Presolved model has 49455 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.53581987e+07  0.00000000e+00  2.72e+04 2.61e+02  9.27e+05     0s
   1   1.26483641e+07 -1.64818160e+07  3.31e+03 2.27e-13  2.11e+05     0s
   2   4.17397643e+06 -5.33655845e+06  3.31e-03 1.14e-13  3.03e+04     0s
   3   4.32

In [2]:
xi_list = GLS_Apr_weekday_PM.xi_list
P = GLS_Apr_weekday_PM.P
L = GLS_Apr_weekday_PM.L  # dimension of xi

314

In [3]:
size(P, 1), size(P, 2)

(56,314)

In [4]:
P

56×314 Array{Float64,2}:
 1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [5]:
using JuMP

mGLSJulia = Model()

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    70022
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    52752

Total number of variables............................:    17640
                     variables with only lower bounds:    17640
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [6]:
getvalue(lam)

56-element Array{Float64,1}:
  357.362
  324.316
  382.677
  537.145
  523.691
  531.967
  184.384
  515.056
  328.343
  459.769
  588.741
  663.157
  614.105
    ⋮    
  727.008
  529.98 
  794.01 
 1658.79 
  206.924
  230.976
  241.779
  519.306
  398.346
  417.469
  576.711
  283.81 

In [7]:
getobjectivevalue(mGLSJulia)

1.0000000000000067

In [8]:
GLS_Apr_weekday_PM.saveDemandVec(getvalue(lam))

In [9]:
getvalue(p)

56×314 Array{Float64,2}:
 0.698184     0.179143     0.122673     …  0.0          0.0        
 7.52855e-16  7.52829e-16  7.52826e-16     0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 1.91377e-15  1.91376e-15  1.91375e-15     1.17028e-15  1.17028e-15
 0.0          0.0          0.0             0.0          0.0        
 6.87666e-16  6.8764e-16   6.87637e-16     0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 ⋮                                      ⋱                          
 0.0          0.0      

In [10]:
route_choice_matrix_Apr_PM = getvalue(p)

56×314 Array{Float64,2}:
 0.698184     0.179143     0.122673     …  0.0          0.0        
 7.52855e-16  7.52829e-16  7.52826e-16     0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 1.91377e-15  1.91376e-15  1.91375e-15     1.17028e-15  1.17028e-15
 0.0          0.0          0.0             0.0          0.0        
 6.87666e-16  6.8764e-16   6.87637e-16     0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 ⋮                                      ⋱                          
 0.0          0.0      

In [11]:
sum(route_choice_matrix_Apr_PM[1, :])

1.0000000000001716

In [12]:
route_choice_matrix_dict_Apr_PM = Dict()

for row_idx = 1:size(route_choice_matrix_Apr_PM)[1]
    for col_idx = 1:size(route_choice_matrix_Apr_PM)[2]
        route_choice_matrix_dict_Apr_PM[(row_idx, col_idx)] = route_choice_matrix_Apr_PM[row_idx, col_idx]
    end
end

In [13]:
outfile = open("../temp_files/route_choice_CDC_Apr_PM.json", "w")

JSON.print(outfile, route_choice_matrix_dict_Apr_PM)

close(outfile)

# Route flow analysis (see [Eq. (9) in M. Hazelton (2000); Page 556])

$\textbf{P}'\textbf{g}$ equals the route flow vector. It seems hard to uniquely determine a route flow vector corresponding to a social link flow vector.